In [27]:
import numpy as np
from collections import defaultdict as dd
from typing import List, Dict, Set
import nltk
from math import log, ceil
import time
import sys
from scipy.sparse import lil_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_distances

In [17]:
import pickle
with open("Dane/wiki_list", "rb") as ifile:
    wiki_list = pickle.load(ifile)

In [3]:
def to_lemma_mapping():
    all_lemmas = dd(list)
    for line in open('Dane/polimorfologik-2.1.txt', encoding='utf-8'):
        L = line.split(';')[:2]
        all_lemmas[L[1].lower()].append(L[0].lower())
    return all_lemmas

In [4]:
with open("Dane/polish.stopwords.txt", "r") as ifile:
    stopwords = [line[:-1] for line in ifile]

In [5]:
stopwords

['a',
 'aby',
 'ach',
 'acz',
 'aczkolwiek',
 'aj',
 'albo',
 'ale',
 'alez',
 'ależ',
 'ani',
 'az',
 'aż',
 'bardziej',
 'bardzo',
 'beda',
 'bedzie',
 'bez',
 'deda',
 'będą',
 'bede',
 'będę',
 'będzie',
 'bo',
 'bowiem',
 'by',
 'byc',
 'być',
 'byl',
 'byla',
 'byli',
 'bylo',
 'byly',
 'był',
 'była',
 'było',
 'były',
 'bynajmniej',
 'cala',
 'cali',
 'caly',
 'cała',
 'cały',
 'ci',
 'cie',
 'ciebie',
 'cię',
 'co',
 'cokolwiek',
 'cos',
 'coś',
 'czasami',
 'czasem',
 'czemu',
 'czy',
 'czyli',
 'daleko',
 'dla',
 'dlaczego',
 'dlatego',
 'do',
 'dobrze',
 'dokad',
 'dokąd',
 'dosc',
 'dość',
 'duzo',
 'dużo',
 'dwa',
 'dwaj',
 'dwie',
 'dwoje',
 'dzis',
 'dzisiaj',
 'dziś',
 'gdy',
 'gdyby',
 'gdyz',
 'gdyż',
 'gdzie',
 'gdziekolwiek',
 'gdzies',
 'gdzieś',
 'go',
 'i',
 'ich',
 'ile',
 'im',
 'inna',
 'inne',
 'inny',
 'innych',
 'iz',
 'iż',
 'ja',
 'jak',
 'jakas',
 'jakaś',
 'jakby',
 'jaki',
 'jakichs',
 'jakichś',
 'jakie',
 'jakis',
 'jakiś',
 'jakiz',
 'jakiż',
 'jak

In [6]:
lemmas = to_lemma_mapping()

In [7]:
lemmas["zielonej"]

['zielona', 'zielony']

In [8]:
wiki_list[0]

('AWK',
 [['AWK'],
  ['AWK',
   '–',
   'interpretowany',
   'język',
   'programowania',
   ',',
   'którego',
   'główną',
   'funkcją',
   'jest',
   'wyszukiwanie',
   'i',
   'przetwarzanie',
   'wzorców',
   'w',
   'plikach',
   'lub',
   'strumieniach',
   'danych',
   '.',
   'Jest',
   'także',
   'nazwą',
   'programu',
   'początkowo',
   'dostępnego',
   'dla',
   'systemów',
   'operacyjnych',
   'będących',
   'pochodnymi',
   'UNIX-a',
   ',',
   'obecnie',
   'także',
   'na',
   'inne',
   'platformy',
   '.'],
  ['AWK',
   'jest',
   'językiem',
   ',',
   'który',
   'w',
   'znacznym',
   'stopniu',
   'wykorzystuje',
   'tablice',
   'asocjacyjne',
   ',',
   'stringi',
   'i',
   'wyrażenia',
   'regularne',
   '.',
   'Nazwa',
   'języka',
   'pochodzi',
   'od',
   'pierwszych',
   'liter',
   'nazwisk',
   'jego',
   'autorów',
   'Alfreda',
   'V.',
   'Aho',
   ',',
   'Petera',
   'Weinbergera',
   'i',
   'Briana',
   'Kernighana',
   'i',
   'czasami',
  

In [9]:
test = dd(lambda:0)

In [10]:
"a" in test

False

In [19]:
corpus = []
for title, article in wiki_list:
    art_processed = []
    for line in article:
        for w in line:
            w = w.lower()
            if w in lemmas:
                art_processed.extend(lemmas[w])
            else:
                art_processed.append(w)
    corpus.append(" ".join(art_processed))

In [13]:
corpus = []
key = ""
with open("Dane/fp_wiki.txt") as f:
    cntr = -1
    for line in f:
        tokenized_line = line.split(": ")
        if tokenized_line[0] == "TITLE":
            cntr+=1
            corpus.append([])
            continue
        corpus[-1].append(line)

In [15]:
corpus = ["".join(el) for el in corpus]

In [20]:
corpus[:3]

['awk awk – interpretować język programować , który główny funkcja być wyszukiwać i przetwarzanie przetwarzać wzorzec wat w wiek plik plika luba lub lubić strumień strumień dana dane dany dać . być także nazwa nazwać program początkowo początkowy dostępny dla system operacyjny być pochodna pochodny unix , obecnie także na inny platforma . awk być język , który wat w wiek znaczny stopień wykorzystywać tablica tablice asocjacyjny , stringi i wyrazić wyrażenie regularny . nazwa język pochodzić oda od oda pierwsza pierwsze pierwszy litera litra nazwisko on autor alfred alfreda alfredo v. aho , peter weinbergera i brian kernighana i czas być zapisywać mały mała małe mały litera oraz odczytywać jaka jako jeden jedno słowo `` awk `` .',
 'alergologia alergologia – dziedzina medycyna zajmować zajmujący się rozpoznawać i leczenie leczyć schorzenie alergiczny .',
 'ascii ascii ( czyta czytaj . `` aski ; `` angielski . `` american standard code for information interchange `` ) – 7-bitowy kod koda

In [11]:
def process_word(w):
    w = w.lower()
    if w in lemmas:
        return " ".join(lemmas[w])
    return w

In [21]:
vectorizer = TfidfVectorizer(stop_words = stopwords)

In [23]:
X = vectorizer.fit_transform(corpus)

In [31]:
from sklearn.neighbors import NearestNeighbors
nbrs = NearestNeighbors(n_neighbors=10, algorithm='ball_tree', metric=cosine_distances, n_jobs=50).fit(X)


/home/mikolaj/miniconda3/envs/IoT-Pathway/lib/python3.7/site-packages/sklearn/neighbors/_base.py:414: UserWarning: cannot use tree with sparse input: using brute force
  warnings.warn("cannot use tree with sparse input: "


In [ ]:
distances, indices = nbrs.kneighbors(X)

In [ ]:
import pickle
with open("Dane/indices.pickle", "wb") as ofile:
    pickle.dump(indices, ofile)

In [ ]:
"test"